![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from talib import RSI
from datetime import datetime as dt
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from AlgorithmImports import *
idx = pd.IndexSlice
qb = QuantBook()


In [14]:
qb.SetStartDate(2016, 1, 1)
qb.SetEndDate(2018, 1, 1)
history = qb.History(['SPY'], 360, Resolution.Daily)
history.tail()

close        high         low        open  \
symbol           time                                                         
SPY R735QTJ8XC9X 2015-12-25  179.938218  180.533200  179.745724  179.999466   
                 2015-12-29  179.561980  179.588229  178.450764  179.246989   
                 2015-12-30  181.469421  181.810660  180.655696  180.708194   
                 2015-12-31  180.183210  181.303176  180.034465  181.198179   
                 2016-01-01  178.398265  180.148212  178.398265  179.518231   

                                  volume  
symbol           time                     
SPY R735QTJ8XC9X 2015-12-25   42633411.0  
                 2015-12-29   69733465.0  
                 2015-12-30   94270993.0  
                 2015-12-31   64749212.0  
                 2016-01-01  101216811.0